In [2]:
%matplotlib inline

import os
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from datetime import datetime as dt
if int(os.environ.get("MODERN_PANDAS_EPUB", 0)):
    import prep # noqa

pd.options.display.max_rows = 10
pd.options.display.max_columns = 30
sns.set(style='ticks', context='talk') 

In [3]:
#Reading data from the CSV File.
auto = pd.read_csv('data/Auto.csv')
auto.head()

,customer_id,Customer,Gender,Income,Employment Status,Education,Marital Status,Country,State Code,Location Type,Policy Name,Policy Type,Number of Policies,Premium Per Month,Coverage,Sales Channel,Vehicle Type,Vehicle Size,date,Claim Reason,Claim Demanded,Claim Sanctioned,Number of Open Complaints,Response,Months Since Last Claim,Months Since Policy Inspection
0,0,BU79786,F,56274,Employed,Bachelor,Married,US,KS,Suburban,Corporate L3,Corporate Auto,1,69,Basic,Agent,Two-Door Car,Medsize,2/24/11,Collision,276.351928,384.811147,0,No,32,5
1,1,QZ44356,F,0,Unemployed,Bachelor,Single,US,NE,Suburban,Personal L3,Personal Auto,8,94,Extended,Agent,Four-Door Car,Medsize,1/31/11,Scratch/Dent,697.953590,1131.464935,0,No,13,42
2,2,AI49188,F,48767,Employed,Bachelor,Married,US,OK,Suburban,Personal L3,Personal Auto,2,108,Premium,Agent,Two-Door Car,Medsize,2/19/11,Collision,1288.743165,566.472247,0,No,18,38
3,3,WW63253,M,0,Unemployed,Bachelor,Married,US,MO,Suburban,Corporate L2,Corporate Auto,7,106,Basic,Call Center,SUV,Medsize,1/20/11,Collision,764.586183,529.881344,0,No,18,65
4,4,HB64268,M,43836,Employed,Bachelor,Single,US,KS,Rural,Personal L1,Personal Auto,1,73,Basic,Agent,Four-Door Car,Medsize,2/3/11,Collision,281.369258,138.130879,0,No,12,44


In [4]:
column_names = {'Premium Per Month': 'Premium_Per_Month','Sales Channel':'Sales_Channel','Employment Status':'Employment_Status',
               'Marital Status':'Marital_Status','State Code':'State_Code','Location Type':'Location_Type','Policy Name':'Policy_Name',
               'Policy Type':'Policy_Type','Number of Policies':'Number_of_Policies','Claim Reason':'Claim_Reason','Claim Demanded':'Claim_Demanded',
               'Claim Sanctioned':'Claim_Sanctioned','Vehicle Type':'Vehicle_Type','Vehicle Size':'Vehicle_Size','Number of Open Complaints':'Number_of_Open_Complaints',
               'Months Since Last Claim':'Months_Since_Last_Claim','Months Since Policy Inspection':'Months_Since_Policy_Inspection'}
auto = auto.rename(columns=column_names)
auto.head()

,customer_id,Customer,Gender,Income,Employment_Status,Education,Marital_Status,Country,State_Code,Location_Type,Policy_Name,Policy_Type,Number_of_Policies,Premium_Per_Month,Coverage,Sales_Channel,Vehicle_Type,Vehicle_Size,date,Claim_Reason,Claim_Demanded,Claim_Sanctioned,Number_of_Open_Complaints,Response,Months_Since_Last_Claim,Months_Since_Policy_Inspection
0,0,BU79786,F,56274,Employed,Bachelor,Married,US,KS,Suburban,Corporate L3,Corporate Auto,1,69,Basic,Agent,Two-Door Car,Medsize,2/24/11,Collision,276.351928,384.811147,0,No,32,5
1,1,QZ44356,F,0,Unemployed,Bachelor,Single,US,NE,Suburban,Personal L3,Personal Auto,8,94,Extended,Agent,Four-Door Car,Medsize,1/31/11,Scratch/Dent,697.953590,1131.464935,0,No,13,42
2,2,AI49188,F,48767,Employed,Bachelor,Married,US,OK,Suburban,Personal L3,Personal Auto,2,108,Premium,Agent,Two-Door Car,Medsize,2/19/11,Collision,1288.743165,566.472247,0,No,18,38
3,3,WW63253,M,0,Unemployed,Bachelor,Married,US,MO,Suburban,Corporate L2,Corporate Auto,7,106,Basic,Call Center,SUV,Medsize,1/20/11,Collision,764.586183,529.881344,0,No,18,65
4,4,HB64268,M,43836,Employed,Bachelor,Single,US,KS,Rural,Personal L1,Personal Auto,1,73,Basic,Agent,Four-Door Car,Medsize,2/3/11,Collision,281.369258,138.130879,0,No,12,44


Question 1: Can we predict Number of Policies sold using Sales Channel?

In [5]:
auto1 = auto.groupby('Sales_Channel').Number_of_Policies.sum()

In [6]:
auto1.head()

Sales_Channel
Agent          10247
Branch          7519
Call Center     5306
Web             4021
Name: Number_of_Policies, dtype: int64

In [7]:
import scipy.stats as stats
stats.chi2_contingency(auto1,correction=False)

(0.0, 1.0, 0, array([ 10247.,   7519.,   5306.,   4021.]))

In [8]:
agent = auto[auto['Sales_Channel']=='Agent']['Number_of_Policies']
branch = auto[auto['Sales_Channel']=='Branch']['Number_of_Policies']
call_center = auto[auto['Sales_Channel']=='Call Center']['Number_of_Policies']
web = auto[auto['Sales_Channel']=='Web']['Number_of_Policies']

In [9]:
stats.ttest_ind(agent,branch)

Ttest_indResult(statistic=0.29071034788636574, pvalue=0.771282834547242)

In [10]:
import statsmodels.formula.api as smf
lm = smf.ols(formula='Number_of_Policies ~ Sales_Channel',data=auto).fit()
lm.params

Intercept                       2.947081
Sales_Channel[T.Branch]        -0.017981
Sales_Channel[T.Call Center]    0.059151
Sales_Channel[T.Web]            0.087636
dtype: float64

In [11]:
lm.pvalues

Intercept                       0.000000
Sales_Channel[T.Branch]         0.772528
Sales_Channel[T.Call Center]    0.397165
Sales_Channel[T.Web]            0.256140
dtype: float64

From the above results we can interpret that number of policies cannot be predicted using Sales Channel as no p-values is above the threshold.

Question 2: Is Employment Status and Premium per month correlated?

In [12]:
auto2 = pd.DataFrame(columns =['Customer_ID','Employment','Premium','Employments'])
auto2['Customer_ID'] = auto['customer_id']
auto2['Employment'] = auto['Employment_Status']
auto2['Premium'] = auto['Premium_Per_Month']
auto2.head()

,Customer_ID,Employment,Premium,Employments
0,0,Employed,69,NaN
1,1,Unemployed,94,NaN
2,2,Employed,108,NaN
3,3,Unemployed,106,NaN
4,4,Employed,73,NaN


In [13]:
for i in auto2['Customer_ID']:
    if auto2.loc[i,'Employment'] == "Employed":
        auto2.loc[i,'Employments'] = 1
    else:
        auto2.loc[i,'Employments'] = 0
  

In [14]:
auto21 = auto2.groupby('Employments').Premium.sum()
auto21.head()

Employments
0    320339
1    531126
Name: Premium, dtype: int64

In [15]:
stats.chi2_contingency(auto21,correction=False)

(0.0, 1.0, 0, array([ 320339.,  531126.]))

In [16]:
employed = auto2[auto2['Employments']== 1]['Premium']
unemployed = auto2[auto2['Employments']== 0]['Premium']

In [17]:
stats.ttest_ind(employed,unemployed)

Ttest_indResult(statistic=-0.023550236706379877, pvalue=0.9812118810291357)

In [18]:
auto2['Premium']=np.int64(auto2['Premium'])
auto2['Employments'] = np.int64(auto2['Employments'])
auto2['Premium'].corr(auto2['Employments'])

-0.00024644063095685724

From the above results it can be interpreted that employment status is not correlated to premium per month

Question 3: Is Marital Status and Premium Per Month correlated?

In [20]:
auto3 = pd.DataFrame(columns =['Customer_ID','Married','Premium','Marrieds'])
auto3['Customer_ID'] = auto['customer_id']
auto3['Married'] = auto['Marital_Status']
auto3['Premium'] = auto['Premium_Per_Month']
auto3.head()

,Customer_ID,Married,Premium,Marrieds
0,0,Married,69,NaN
1,1,Single,94,NaN
2,2,Married,108,NaN
3,3,Married,106,NaN
4,4,Single,73,NaN


In [21]:
for i in auto3['Customer_ID']:
    if auto3.loc[i,'Married'] == "Married":
        auto3.loc[i,'Marrieds'] = 1
    else:
        auto3.loc[i,'Marrieds'] = 0

auto31 = auto3.groupby('Marrieds').Premium.sum()
auto31.head()

Marrieds
0    358387
1    493078
Name: Premium, dtype: int64

In [22]:
stats.chi2_contingency(auto31,correction=False)

(0.0, 1.0, 0, array([ 358387.,  493078.]))

In [24]:
married = auto3[auto3['Marrieds']== 1]['Premium']
nonmarried = auto3[auto3['Marrieds']== 0]['Premium']
stats.ttest_ind(married,nonmarried)

Ttest_indResult(statistic=-0.49153333020070561, pvalue=0.62306109790049313)

In [25]:
auto3['Premium']=np.int64(auto3['Premium'])
auto3['Marrieds'] = np.int64(auto3['Marrieds'])
auto3['Premium'].corr(auto3['Marrieds'])

-0.0051435655121088329

From the above results it can be interpreted that marital status is not correlated to premium per month